### Below, is code for creating the VGG16 model from scratch using Keras as well as the training weights for the Vgg16 model made available by the researchers.

In [50]:
%matplotlib inline

# path = "dogscats/"
path = "dogscats/sample/"

from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

# Utils holds convenient functions needed to run this ipynb
# reload(utils) allows for updating the notebook with any changes
# made to the library
import utils; reload(utils)
# importing the plots function in utils
from utils import plots

from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

### The next step is to retrieve all the classes that the VGG16 model contains so that the classes can then be displayed properly.

In [57]:
# This is the path of the file which contains the class names in dictionary format.
# FILEPATH = 'http://www.platform.ai/models/'; CLASS_FILE= 'imagenet_class_index.json'

# fpath = get_file(CLASS_FILE, FILEPATH+CLASS_FILE, cache_subdir='models')

# # The file is opened and the json data is loaded as class_dict
# with open(fpath) as f:class_dict = json.load(f)

FILES_PATH = 'http://www.platform.ai/models/'; CLASS_FILE='imagenet_class_index.json'
# Keras' get_file() is a handy function that downloads files, and caches them for re-use later
fpath = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')
with open(fpath) as f: class_dict = json.load(f)
# Convert dictionary with string indexes into an array
classes = [class_dict[str(i)][1] for i in range(len(class_dict))]
    
print(type(classes))

# print(len(class_dict))

# print class_dict.keys()[0]
# print class_dict.values()[0]
# # Basically, the item is a tuple that has an index number that maps to
# # an array that holds the id of the item in the 0th position & the actual class in the first position
# print class_dict.items()[0]

# print str(0)
# sth = class_dict[str(0)] 
# print sth
# print sth[1]


<type 'list'>


In [59]:
# As shown above, the class_dictionary is accessed to retrieve all the classes in the form of an array
# classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

print(classes[:5])

[u'tench', u'goldfish', u'great_white_shark', u'tiger_shark', u'hammerhead']


## This is a pretty important step.

### Below, is how the building blocks of the model are created. By building blocks, I mean the types of layers that the model consists. In this case, the layers are convolutional layers and fully connected layers.

In [60]:
def ConvolutionalLayer(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D(2,2), strides=(2,2))

In [61]:
def FullyConnectedLayer(model):
    model.add(Dense (4096, activation='relu'))
    model.add(Dropout(0.5))

### The next step is a little nuanced, and I am still not completely sure on why this is necessary, but it has something to do with how the RGB channels  are used in training the VGG model. 

- When the model was trained by the original researchers, their deep learning library accepted color channels in the form of BGR.
- Python accepts color channels in the form of RGB, so that needs to be accounted for.

In [62]:
# The mean of each BGR channel is provided by researchers.
# A reshaped 3X1 array is returned.
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean
    # Reversing the order of the vgg_mean to make it acceptable for PYTHON
    return x[:, ::-1]


In [63]:
def Vgg_Scratch():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))
    
    ConvolutionalLayer(2, model, 64)
    ConvolutionalLayer(2, model, 128)
    ConvolutionalLayer(3, model, 256)
    ConvolutionalLayer(3, model, 512)
    ConvolutionalLayer(3, model, 512)
    
    model.add(Flatten())
    FullyConnectedLayer(model)
    FullyConnectedLayer(model)
    model.add(Dense(1000, activation='softmax'))
    return model   


In [64]:
model = Vgg_Scratch()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


TypeError: 'int' object is not iterable